In [1]:
!pip install datasets
!pip install sklearn
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')
from datasets import load_dataset, load_from_disk
import os
import pickle
from tqdm import tqdm
import pandas as pd
# Data processing
import numpy as np# Visualization
import matplotlib.pyplot as plt
import seaborn as sns# Dataset
from sklearn import datasets# Dimensionality reduction
from sklearn.decomposition import PCA# Modeling
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
import scipy.cluster.hierarchy as hier
from sklearn.mixture import GaussianMixture# Number of clusters
from sklearn.metrics import silhouette_score
from datasets import load_dataset, load_from_disk
import os
import pickle
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split


type_to_t = {
    'disorder': 'T-11',
    'procedure': 'T-39',
    'finding': 'T-18',
    'substance': 'T-55'
}

with open('drive/MyDrive/Tesi_LossAcc/id2tkn.pickle', 'rb') as handle:
    id2tkn = pickle.load(handle)

data_path = 'drive/MyDrive/Tesi_LossAcc/annotations_stream_phase2_v1_1d_256_ALL_TYPES_prepared_split'
cui_types = 'disorder,procedure,finding,substance'
#cui_types = 'disorder'
FOLDER = "drive/MyDrive/Tesi_LossAcc/[foresight_small]-[types={}]".format(cui_types)
subset = 8000

cui_types = cui_types.split(',')
def map_to_df(data_path, mode, cui_types, id2tkn):
    dataset = load_from_disk(os.path.join(data_path, mode))
    cui_types = [type_to_t[t] for t in cui_types] + ['sep']

    df = []
    for sample in tqdm(dataset):
        patient_id = sample['patient_id']
        ts = 0
        tokens = []
        for token_type, token in zip(sample['token_type'], sample['input_ids']):
            if token_type not in cui_types: continue

            if token_type == 'sep':
                ts += 1
            else:
                if token not in tokens:
                    df.append([patient_id, token_type, id2tkn[token], ts])
                    tokens.append(token)
    df = pd.DataFrame(df)
    return df
df_train = map_to_df(data_path, 'train', cui_types, id2tkn)
df_test = map_to_df(data_path, 'test', cui_types, id2tkn)

cols = df_train.columns
df_train.rename (columns = {cols[0]:'patient_id',cols[1]:'concept_types',cols[2]:'concept',cols[3]:'timestamp'}, inplace=True)
cols_2 = df_test.columns
df_test.rename (columns = {cols_2[0]:'patient_id',cols_2[1]:'concept_types',cols_2[2]:'concept',cols_2[3]:'timestamp'}, inplace=True)

df_train = df_train.drop_duplicates()
df_test = df_test.drop_duplicates()


#Remove patient with a single timestamp value
patient_count_train = df_train.groupby("patient_id").agg({"timestamp": "nunique"})
patient_count_train = patient_count_train[(patient_count_train["timestamp"] == 1)] #& (patient_count_t["concept"] == 1)]
df_train = df_train[~df_train['patient_id'].isin(patient_count_train.index)]

patient_count_t = df_test.groupby("patient_id").agg({"timestamp": "nunique"})
patient_count_t = patient_count_t[(patient_count_t["timestamp"] == 1)] #& (patient_count_t["concept"] == 1)]
df_test = df_test[~df_test['patient_id'].isin(patient_count_t.index)]

# Concatena df_train e df_test
new_df = pd.concat([df_train, df_test], ignore_index=True)
"""#GROUND TRUTH GENERATION
time = 1
#Creazione dataframe contenente il numero di timestamp unici per patient_id
df_timestamp_unic = new_df.groupby('patient_id')['timestamp'].nunique().reset_index()
df_timestamp_unic.columns = ['patient_id', 'timestamp_unic']
#Generazione ground_truth
ground_truth = pd.DataFrame() # creazione del DataFrame vuoto
for patient_id in df_timestamp_unic['patient_id'].unique():
    timestamp_unic = df_timestamp_unic.loc[df_timestamp_unic['patient_id'] == patient_id, 'timestamp_unic'].iloc[0]
    if timestamp_unic > time:
        group = new_df.loc[new_df['patient_id'] == patient_id] # selezione del gruppo di righe per il paziente corrente
        unique_timestamps = group['timestamp'].unique()
        max_timestamps = pd.Series(unique_timestamps).nlargest(time)
        ground_truth = pd.concat([ground_truth, group.loc[group['timestamp'].isin(max_timestamps)]])
    elif timestamp_unic <= time:
        group = new_df.loc[(new_df['patient_id'] == patient_id) & (new_df['timestamp'] != 0)] # selezione del gruppo di righe per il paziente corrente, escludendo i timestamp uguali a zero
        ground_truth = pd.concat([ground_truth, group])
ground_truth.reset_index(drop=True, inplace=True) # reset dell'indice del DataFrame"""
# Load ground truth file
ground_truth = pd.read_csv('/content/drive/MyDrive/Tesi_LossAcc/ground_truthTimestampDF.csv')
# FILTERING
# Remove of all concepts other than T-11 in ground truth
ground_truth = ground_truth[ground_truth['concept_types'] == 'T-11']
new_df['patient_id'] = new_df['patient_id'].astype('int64')
new_df['concept'] = new_df['concept'].astype('int64')

merged_test_time = pd.merge(new_df, ground_truth, on=["patient_id", "concept", "concept_types", "timestamp"], how="outer", indicator=True)
filtered_test_time = merged_test_time[merged_test_time["_merge"] == "left_only"]
new_df1 = filtered_test_time.drop("_merge", axis=1)

unique_sources = ground_truth['patient_id'].unique()

new_df2 = new_df1[new_df1['patient_id'].isin(unique_sources)]
new_df2 = new_df2.copy()
new_df2['patient_id'] = new_df2['patient_id'].astype('uint32')
new_df2['concept'] = new_df2['concept'].astype('uint32')
new_df2['timestamp'] = new_df2['timestamp'].astype('uint32')
# Removal of previously encountered disorders
common_concepts = pd.merge(ground_truth, new_df2, on=['patient_id', 'concept'])

common_patient_ids = common_concepts['patient_id'].unique()
common_concepts = common_concepts[['patient_id', 'concept']]
filtered_ground_truth = ground_truth[~((ground_truth['patient_id'].isin(common_concepts['patient_id'])) & (ground_truth['concept'].isin(common_concepts['concept'])))]

ground_truth_filtered = ground_truth[ground_truth['patient_id'].isin(filtered_ground_truth['patient_id'])]
unique_sources = ground_truth_filtered['patient_id'].unique()

new_df = new_df2[new_df2['patient_id'].isin(unique_sources)]
all_df = new_df
all_time = all_df[all_df['patient_id'].isin(unique_sources)]
num_all = all_time['patient_id'].nunique()
all_time_disorder = all_time[all_time['concept_types'] == 'T-11'].copy()
conteggio_occorrenze = all_time_disorder.groupby(all_time_disorder.columns.tolist(),as_index=False).size()
cols_prova = conteggio_occorrenze.columns
conteggio_occorrenze.rename(columns = {cols_prova[4]:'occurrence'}, inplace=True)
dis_clust = (conteggio_occorrenze.groupby('patient_id')['concept'].value_counts().unstack().fillna(0)).reset_index()
dis_clust2 = dis_clust.set_index('patient_id')

# LABEL GENERATION
label_vect = (ground_truth.groupby('patient_id')['concept'].value_counts().unstack().fillna(0)).reset_index()
label_vect.rename(columns={'patient_id': 'source'}, inplace=True)
#label_vect.to_csv("/content/drive/MyDrive/Tesi_LossAcc/LabelsLess1364.csv")

# FEATURES GENERATION
feat_vect = (new_df2.groupby('patient_id')['concept'].value_counts().unstack().fillna(0)).reset_index()
feat_vect.rename(columns={'patient_id': 'source'}, inplace=True)
# feat_vect.to_csv("/content/drive/MyDrive/Tesi_LossAcc/FeatLessPat.csv")

# TRUNCATEDSVD PCA
X = dis_clust2.values
tsvd = TruncatedSVD(n_components=X.shape[1])
X_tsvd = tsvd.fit_transform(X)

n_components=X.shape[1]

variance_cumulative = np.cumsum(tsvd.explained_variance_ratio_)

"""# Plot
plt.plot(variance_cumulative)
plt.xlabel('Numero di componenti')
plt.ylabel('Varianza cumulativa spiegata')
plt.show()"""

transformed_data_reduced = X_tsvd[:,:n_components]

df_pcas = {}
n_components_dict = {}
variance_percentages = [50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]

for p in variance_percentages:
    n_components = np.argmax(variance_cumulative >= p/100) + 1
    n_components_dict[p] = n_components
    transformed_data_reduced = X_tsvd[:,:n_components]
    df_pca = pd.DataFrame(transformed_data_reduced, columns=[f'PC{i+1}' for i in range(n_components)])
    df_pca.index = dis_clust2.index
    df_pcas[p] = df_pca

df_pca = df_pcas[80]
print("The number of components for the selected variance is:",n_components_dict[80])

#KMEANS
n_clusters = 2685
kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init=10)

df_pca_copy = df_pca.copy()
df_pca_copy['cluster'] = kmeans.fit_predict(df_pca_copy)
df_copy = dis_clust2.copy()
df_copy.reset_index(inplace=True)
df_pca_copy.reset_index(inplace=True)
df_pca_copy["patient_id"] = df_pca_copy["patient_id"].astype(int)
df_copy["patient_id"] = df_copy["patient_id"].astype(int)
df_copy['cluster'] = df_copy['patient_id'].map(dict(zip(df_pca_copy['patient_id'],df_pca_copy['cluster'])))
grouped = df_copy.groupby("cluster")

#EDGES GENERATION
# Count the number of occurrences of each cluster
cluster_counts = df_copy['cluster'].value_counts()
# Filter clusters with only one patient_id
single_patient_clusters = cluster_counts[cluster_counts == 1].index.tolist()
# Filter the dataframe to include only the single patient clusters
df_single_patient_clusters = df_copy[df_copy['cluster'].isin(single_patient_clusters)]
df_primiEdge = df_copy[['patient_id', 'cluster']].copy()
merge_clust = df_primiEdge.copy()
merge_clust = merge_clust.merge(df_primiEdge, on='cluster')
merge_clust.drop_duplicates(inplace=True)
#Removing self-connection
merge_clust = merge_clust[merge_clust['patient_id_x'] != merge_clust['patient_id_y']]
edge_cluster = merge_clust
# Replace the 'cluster' column with the 'label' column and assign the value 'has_common'
edge_cluster['label'] = edge_cluster['cluster'].map(lambda x: 'has_common')
# Remove the 'common_concept' column
edge_cluster = edge_cluster.drop('cluster', axis=1)
edge_cluster = edge_cluster.rename(columns={'patient_id_x': 'source'})
edge_cluster = edge_cluster.rename(columns={'patient_id_y': 'target'})
#edge_cluster.to_csv("/content/drive/MyDrive/Tesi_LossAcc/EdgeGraph_Clustered2685.csv")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 6.4 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Mounted at /content/drive


100%|██████████| 2539/2539 [00:01<00:00, 2023.55it/s]


The number of components for the selected variance is: 239
